In [69]:
! pip install -U google.ai.generativelanguage

In [70]:
! pip install -U google-auth-oauthlib

In [71]:
! pip install google-genai

In [72]:
import json
from google import genai
from google.genai import types
# Only run this block for Google AI API
with open("api_key.env", 'r') as f:
            api_key = f.read()
client = genai.Client(api_key=api_key)

In [73]:
service_account_file_name = 'service_account_key.json'

from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(service_account_file_name)

scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/generative-language.retriever'])


In [74]:
! pip install google-labs-html-chunker

In [82]:
from gemini_aqa_client import GeminiAQA, Document

In [81]:
aqa = GeminiAQA()
lds = aqa.delete_corpus("corpora/lds-3jy6n1g0gn9r")
# print(aqa.list_corpora())




FailedPrecondition: 400 Cannot delete non-empty Corpus

In [85]:
for document in lds.list_documents():
    doc = Document(document.name, lds)
    for chunk in doc.list_chunks():
        doc.delete_chunk(chunk.name)
        print(chunk.name)
    lds.delete_document(document.name)

In [87]:
doc = lds.create_document("BookOfMormon", metadata={"book":"Book of Mormon"})



In [ ]:

doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=639") # start
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=2202") # 3 witnesses
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=3882") # 8 witnesses
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=4872") # 1 nephi
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=154527") # 2 nephi
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=332408") # jacob
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=386619") # enos
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=393452") # jarom
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=397940") # omni
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=406275") # WOM 
doc.ingest_url("https://quod.lib.umich.edu/cgi/m/mormon/mormon-idx?type=DIV0&byte=411709") # Mosiah

In [183]:
question = "Why did Nephi marry before getting the brass plates? He had many children and some grew to be prophets, but I’m unsure why he married during his journey, please cite verses"
prompt = """
I am going to ask a question. Your purpose is to ask an AQA model (a model that will actually get the information, in an accurate collection) various questions.

Give at least 5 questions to send to AQA model for checking and collecting. Questions should be simple, and prefer, but not depended only on, closed-ended question, given to find facts that will be depended on later to answer the question at hand 

Also, devote one more question to cite information

Only provide the questions, in json format

The Context of this question is an LDS Search, where it is search church documents, especially ones like the Book Of Mormon

Respond in json format, in an array, with each question being a part of the array

ONLY MAKE QUESTIONS related to the topic at hand, for example, if the question is 'what is an apple', questions give would be 'what is the genus of the apple', or 'where is an apple from', or 'is an apple a fruit?'

the format will have each question in the array be a dict, having the key of 'question'

Also, grounding in some scriptural context could be great. Further questions can be asked for quotes for applicable scriptures.
question : 
"""
response = client.models.generate_content(
    model="gemini-1.5-flash-latest", 
    contents=prompt + question, 
    config=types.GenerateContentConfig(
    response_mime_type= 'application/json'))
array = json.loads(response.text)
print(array)



[{'question': 'What is the age of Nephi when he receives the command to obtain the plates of brass, according to the Book of Mormon?'}, {'question': 'Does the Book of Mormon specify the length of time Nephi and his family spent traveling from Jerusalem to their new settlement in the wilderness?'}, {'question': "Does the text describe the circumstances surrounding Nephi's marriage, including details about his wife's name or family background?"}, {'question': "Are there any verses in the Book of Mormon that indicate Nephi's wife accompanied him on the journey to obtain the brass plates?"}, {'question': 'How many children did Nephi have, and at what points in his life did he have them, according to the Book of Mormon text?'}, {'question': "What are the specific verses in the Book of Mormon that describe Nephi's marriage and family life during his journey to obtain the brass plates, and his children becoming prophets?"}]


In [184]:
full_answer = ""
for q in array:
    answer = lds.generate_answer(q["question"], "EXTRACTIVE")

    full_answer += "Answer to " + q["question"] + " is: '" + answer.answer.content.parts[0].text + "' With a confidence of: " + str(answer.answerable_probability) + " \n\n\n"
print(full_answer)

Answer to What is the age of Nephi when he receives the command to obtain the plates of brass, according to the Book of Mormon? is: 'I also knew that the law was engraven upon the plates of brass' With a confidence of: 0.1374151110649109 


Answer to Does the Book of Mormon specify the length of time Nephi and his family spent traveling from Jerusalem to their new settlement in the wilderness? is: 'many days' With a confidence of: 0.9928644895553589 


Answer to Does the text describe the circumstances surrounding Nephi's marriage, including details about his wife's name or family background? is: 'one of the daughters of Ishmael' With a confidence of: 0.983169674873352 


Answer to Are there any verses in the Book of Mormon that indicate Nephi's wife accompanied him on the journey to obtain the brass plates? is: 'And I, Nephi, had also brought the records which were engraven upon the plates of brass; and also the ball, or compass, which was prepared for my father by the hand of the Lor

In [185]:
prompt = """
You will be given a few paragraphs of information answering the question, with an AI giving additional questions to answer the original one.
""" + question + """
Respond in a sentence or two using the information given from reliable sources to answer the question. 

Avoid responses with lower confidence eg 0.01 to higher ones 0.99

If you want to go past one or two sentences you can, just have information central to the question availible. 

For example, you could cite the answer (to the scripture chapter, verse, book title, etc,), or you could provide further context or facts. 

If you want to go further, you can give some quotations of scripture if they are given

Rely ONLY on the q&a unless you are extremely confident you can add more. 
here is the Q&A:

Also, respond to the original question, as if you came up with the questions and answers to the Q&A, so like no "based on the provided answers" or anything like that.

dont add any text effects
""" + full_answer

response = client.models.generate_content(
    model="gemini-1.5-flash-latest", 
    contents=prompt
)
print(response.text)

Nephi married one of the daughters of Ishmael during his journey,  as indicated by the Book of Mormon.  While specific verses detailing the timing of his marriage relative to obtaining the brass plates aren't definitively provided,  the context suggests it happened before or during the journey  given that he had children who later became prophets.



In [ ]:


# q = "Are there any additional documents associated with the Book of Mormon, that provide further detail or clarification about the experience of Oliver Cowdery, David Whitmer, and Martin Harris as witnesses?"
# answer = lds.generate_answer(q, "VERBOSE")
# print(answer.answer.content.parts[0].text)
# print("confidence: " + str(answer.answerable_probability))

The source provided, "The Book of Mormon", does not contain any additional documents associated with the Book of Mormon that provide further detail or clarification about the experience of Oliver Cowdery, David Whitmer, and Martin Harris as witnesses.
confidence: 0.0004242501745466143
